In [3]:
%load_ext autoreload
%autoreload 2

import os
import sys

import numpy as np
import pandas as pd
import scipy.sparse as sps
import yaml
from itertools import chain, product, combinations
import torch

from time import time as tt
from tqdm import tqdm
sys.path.append("../../../")
from gnn4itk_cf.stages.data_reading.models.trackml_utils import *

from gnn4itk_cf.stages.data_reading.data_reading_stage import EventReader
from gnn4itk_cf.stages.data_reading.models.trackml_reader import TrackMLReader

from gnn4itk_cf.stages.graph_construction.models.metric_learning import MetricLearning
from gnn4itk_cf.stages.edge_classifier.models.filter import Filter
from gnn4itk_cf.stages.edge_classifier import InteractionGNN

from gnn4itk_cf.stages.graph_construction.utils import handle_weighting
from gnn4itk_cf.stages.graph_construction.models.utils import graph_intersection, build_edges
from gnn4itk_cf.stages.graph_construction.utils import *

from gnn4itk_cf.stages.track_building import utils 
from torch_geometric.utils import to_scipy_sparse_matrix

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
with open("examples/Example_3/metric_learning_train.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
model_ML = MetricLearning(config)
with open("examples/Example_3/filter_train.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
model_filter = Filter(config)
with open("examples/Example_3/gnn_train.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
model_gnn = InteractionGNN(config)
config = yaml.safe_load(open("examples/Example_3/track_building_eval.yaml", "r"))

In [7]:
model_ML.setup(stage="predict")
dataloaders = model_ML.predict_dataloader()
model_filter.setup('predict')
model_gnn.setup('predict')

Loaded 80 training events, 10 validation events and 10 testing events


/workspace/gnn4itk_cf/utils/loading_utils.py:78: UserWarning: OPTIONAL feature [pid] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")
/workspace/gnn4itk_cf/utils/loading_utils.py:78: UserWarning: OPTIONAL feature [n_hits] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")
/workspace/gnn4itk_cf/utils/loading_utils.py:78: UserWarning: OPTIONAL feature [primary] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")
/workspace/gnn4itk_cf/utils/loading_utils.py:78: UserWarning: OPTIONAL feature [pdg_id] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")
/workspace/gnn4itk_cf/utils/loading_utils.py:78: UserWarning: OPTIONAL feature [ghost] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")
/workspace/gnn4itk_cf/utils/loading_utils.py:78: UserWarning: OPTIONAL feature [shared] not found in data
  warnings.warn(f"OPTIONAL featu

Defining figures of merit
Defining figures of merit


In [8]:
def load_reconstruction_df(graph):
    """Load the reconstructed tracks from a file."""
    pids = torch.zeros(graph.hit_id.shape[0], dtype=torch.int64)
    pids[graph.track_edges[0]] = graph.particle_id
    pids[graph.track_edges[1]] = graph.particle_id

    return pd.DataFrame({"hit_id": graph.hit_id, "track_id": graph.labels, "particle_id": pids})

def load_particles_df(graph):
    """Load the particles from a file."""
    # Get the particle dataframe
    particles_df = pd.DataFrame({"particle_id": graph.particle_id, "pt": graph.pt})

    # Reduce to only unique particle_ids
    particles_df = particles_df.drop_duplicates(subset=['particle_id'])

    return particles_df

def get_matching_df(reconstruction_df, min_track_length=1, min_particle_length=1):
    
    # Get track lengths
    candidate_lengths = reconstruction_df.track_id.value_counts(sort=False)\
        .reset_index().rename(
            columns={"index":"track_id", "track_id": "n_reco_hits"})

    # Get true track lengths
    particle_lengths = reconstruction_df.drop_duplicates(subset=['hit_id']).particle_id.value_counts(sort=False)\
        .reset_index().rename(
            columns={"index":"particle_id", "particle_id": "n_true_hits"})

    spacepoint_matching = reconstruction_df.groupby(['track_id', 'particle_id']).size()\
        .reset_index().rename(columns={0:"n_shared"})

    spacepoint_matching = spacepoint_matching.merge(candidate_lengths, on=['track_id'], how='left')
    spacepoint_matching = spacepoint_matching.merge(particle_lengths, on=['particle_id'], how='left')
    # spacepoint_matching = spacepoint_matching.merge(particles_df, on=['particle_id'], how='left')

    # Filter out tracks with too few shared spacepoints
    spacepoint_matching["is_matchable"] = spacepoint_matching.n_reco_hits >= min_track_length
    spacepoint_matching["is_reconstructable"] = spacepoint_matching.n_true_hits >= min_particle_length

    return spacepoint_matching

def calculate_matching_fraction(spacepoint_matching_df):
    spacepoint_matching_df = spacepoint_matching_df.assign(
        purity_reco=np.true_divide(spacepoint_matching_df.n_shared, spacepoint_matching_df.n_reco_hits))
    spacepoint_matching_df = spacepoint_matching_df.assign(
        eff_true = np.true_divide(spacepoint_matching_df.n_shared, spacepoint_matching_df.n_true_hits))

    return spacepoint_matching_df

def evaluate_labelled_graph(graph, matching_fraction=0.5, matching_style="ATLAS", min_track_length=1, min_particle_length=1):

    if matching_fraction < 0.5:
        raise ValueError("Matching fraction must be >= 0.5")

    if matching_fraction == 0.5:
        # Add a tiny bit of noise to the matching fraction to avoid double-matched tracks
        matching_fraction += 0.00001

    # Load the labelled graphs as reconstructed dataframes
    reconstruction_df = load_reconstruction_df(graph)
    particles_df = load_particles_df(graph)

    # Get matching dataframe
    matching_df = get_matching_df(reconstruction_df, particles_df, min_track_length=min_track_length, min_particle_length=min_particle_length) 
    matching_df["event_id"] = int(graph.event_id)

    # calculate matching fraction
    matching_df = calculate_matching_fraction(matching_df)

    # Run matching depending on the matching style
    if matching_style == "ATLAS":
        matching_df["is_matched"] = matching_df["is_reconstructed"] = matching_df.purity_reco >= matching_fraction
    elif matching_style == "one_way":
        matching_df["is_matched"] = matching_df.purity_reco >= matching_fraction
        matching_df["is_reconstructed"] = matching_df.eff_true >= matching_fraction
    elif matching_style == "two_way":
        matching_df["is_matched"] = matching_df["is_reconstructed"] = (matching_df.purity_reco >= matching_fraction) & (matching_df.eff_true >= matching_fraction)

    return matching_df

In [9]:
def evaluate_labelled_graphs(graphset, config):
    all_y_truth, all_pt  = [], []
    evaluated_events = [
        utils.evaluate_labelled_graph(
            event,
            matching_fraction=config["matching_fraction"],
            matching_style=config["matching_style"],
            min_track_length=config["min_track_length"],
            min_particle_length=config["min_particle_length"],
        )
        for event in tqdm(graphset)
    ]
    evaluated_events = pd.concat(evaluated_events)

    particles = evaluated_events[evaluated_events["is_reconstructable"]]
    reconstructed_particles = particles[particles["is_reconstructed"] & particles["is_matchable"]]
    tracks = evaluated_events[evaluated_events["is_matchable"]]
    matched_tracks = tracks[tracks["is_matched"]]

    n_particles = len(particles.drop_duplicates(subset=['event_id', 'particle_id']))
    n_reconstructed_particles = len(reconstructed_particles.drop_duplicates(subset=['event_id', 'particle_id']))

    n_tracks = len(tracks.drop_duplicates(subset=['event_id', 'track_id']))
    n_matched_tracks = len(matched_tracks.drop_duplicates(subset=['event_id', 'track_id']))

    n_dup_reconstructed_particles = len(reconstructed_particles) - n_reconstructed_particles

    print(f"Number of reconstructed particles: {n_reconstructed_particles}")
    print(f"Number of particles: {n_particles}")
    print(f"Number of matched tracks: {n_matched_tracks}")
    print(f"Number of tracks: {n_tracks}")
    print(f"Number of duplicate reconstructed particles: {n_dup_reconstructed_particles}")   

    # Plot the results across pT and eta
    eff = n_reconstructed_particles / n_particles
    fake_rate = 1 - (n_matched_tracks / n_tracks)
    dup_rate = n_dup_reconstructed_particles / n_reconstructed_particles

    logging.info(f"Efficiency: {eff:.3f}")
    logging.info(f"Fake rate: {fake_rate:.3f}")
    logging.info(f"Duplication rate: {dup_rate:.3f}")
    print(f"Efficiency: {eff:.3f}")
    print(f"Fake rate: {fake_rate:.3f}")
    print(f"Duplication rate: {dup_rate:.3f}")

In [10]:
device ='cuda'
model_ML = model_ML.to("cuda")
model_filter = model_filter.to("cuda")
model_gnn = model_gnn.to("cuda")
for batch in dataloaders[2]:
    batch = batch.to("cuda")
    with torch.no_grad():
        if device == 'cuda':
            with torch.cuda.amp.autocast():
                embedding = model_ML.apply_embedding(batch)
     
    batch.edge_index = build_edges(
        query=embedding, database=embedding, indices=None, r_max=0.1, k_max=10, backend="FRNN"
    )
    R = batch.r**2 + batch.z**2
    flip_edge_mask = R[batch.edge_index[0]] > R[batch.edge_index[1]]
    batch.edge_index[:, flip_edge_mask] = batch.edge_index[:, flip_edge_mask].flip(0)
    with torch.no_grad():
        if device == 'cuda':
            with torch.cuda.amp.autocast():
                out = model_filter(batch)   
    preds = torch.sigmoid(out)
    batch.edge_index = batch.edge_index[:, preds > model_filter.hparams['edge_cut']]
    with torch.no_grad():
        if device == 'cuda':
            with torch.cuda.amp.autocast():
                out = model_gnn(batch)
    batch.scores = torch.sigmoid(out)

    edge_mask = batch.scores > model_gnn.hparams['edge_cut'] 
    # Get number of nodes
    if hasattr(batch, "num_nodes"):
        num_nodes = batch.num_nodes
    elif hasattr(batch, "x"):
        num_nodes = batch.x.size(0)
    elif hasattr(batch, "x_x"):
        num_nodes = batch.x_x.size(0)
    else:
        num_nodes = batch.edge_index.max().item() + 1
    # Convert to sparse scipy array
    sparse_edges = to_scipy_sparse_matrix(
        batch.edge_index[:, edge_mask], num_nodes=num_nodes
    )
    # Run connected components
    candidate_labels = sps.csgraph.connected_components(
        sparse_edges, directed=False, return_labels=True
    )
    batch.labels = torch.from_numpy(candidate_labels[1]).long()
    
    batch.config.append(config)
    print(batch)
    evaluate_labelled_graphs([batch.to('cpu')], config)


DataBatch(cell_val=[15682], geta=[15682], weight=[15682], region=[15682], lx=[15682], lphi=[15682], module_index=[15682], x=[15682], r=[15682], gphi=[15682], hit_id=[15682], lz=[15682], z=[15682], cell_count=[15682], phi=[15682], y=[15682], ly=[15682], leta=[15682], eta=[15682], track_edges=[2, 14278], particle_id=[14278], radius=[14278], nhits=[14278], pt=[14278], config=[2], event_id=[1], num_nodes=15682, batch=[15682], ptr=[2], edge_index=[2, 111209], scores=[111209], labels=[15682])


100%|██████████| 1/1 [00:00<00:00, 17.73it/s]

Number of reconstructed particles: 1
Number of particles: 1386
Number of matched tracks: 1
Number of tracks: 98
Number of duplicate reconstructed particles: 0
Efficiency: 0.001
Fake rate: 0.990
Duplication rate: 0.000


DataBatch(cell_val=[14296], geta=[14296], weight=[14296], region=[14296], lx=[14296], lphi=[14296], module_index=[14296], x=[14296], r=[14296], gphi=[14296], hit_id=[14296], lz=[14296], z=[14296], cell_count=[14296], phi=[14296], y=[14296], ly=[14296], leta=[14296], eta=[14296], track_edges=[2, 13029], particle_id=[13029], radius=[13029], nhits=[13029], pt=[13029], config=[2], event_id=[1], num_nodes=14296, batch=[14296], ptr=[2], edge_index=[2, 102476], scores=[102476], labels=[14296])


100%|██████████| 1/1 [00:00<00:00, 34.67it/s]

...

100%|██████████| 1/1 [00:00<00:00, 37.28it/s]

Number of reconstructed particles: 3
Number of particles: 1248
Number of matched tracks: 3
Number of tracks: 100
Number of duplicate reconstructed particles: 0
Efficiency: 0.002
Fake rate: 0.970
Duplication rate: 0.000


DataBatch(cell_val=[11896], geta=[11896], weight=[11896], region=[11896], lx=[11896], lphi=[11896], module_index=[11896], x=[11896], r=[11896], gphi=[11896], hit_id=[11896], lz=[11896], z=[11896], cell_count=[11896], phi=[11896], y=[11896], ly=[11896], leta=[11896], eta=[11896], track_edges=[2, 10851], particle_id=[10851], radius=[10851], nhits=[10851], pt=[10851], config=[2], event_id=[1], num_nodes=11896, batch=[11896], ptr=[2], edge_index=[2, 84296], scores=[84296], labels=[11896])


100%|██████████| 1/1 [00:00<00:00, 39.68it/s]

Number of reconstructed particles: 2
Number of particles: 1030
Number of matched tracks: 2
Number of tracks: 98
Number of duplicate reconstructed particles: 0
Efficiency: 0.002
Fake rate: 0.980
Duplication rate: 0.000


In [11]:
export_output = torch.onnx.export(model_filter, sample,"filter.onnx")

RuntimeError: Only tuples, lists and Variables are supported as JIT inputs/outputs. Dictionaries and strings are also accepted, but their usage is not recommended. Here, received an input of unsupported type: Data